In [33]:
import json

input_content_file = 'input-notebook.ipynb'
parsed_json = ''
with open(input_content_file) as json_data:
    parsed_json = json.load(json_data)

template_file = 'dash_template.ipynb'
template_json = ''
with open(template_file) as template_data:
    template_json = json.load(template_data)
    #print(template_json)

In [34]:
text_content = ''
code_content = ''
plot_content = ''
dependencies = []
cells = parsed_json['cells']
for cell in cells:
    #print(cell)
    #print(""               "")
    cell_type = cell['cell_type']
    if cell_type == 'markdown':
        text_cell = cell['source']
        for text_line in text_cell:
            text_content = text_content + text_line
            
    if 'metadata' not in cell:
        raise ValueError("No metadata found")
    if 'tags' not in cell['metadata']:
        raise ValueError("No tags found")

    if cell_type == 'code' and cell['metadata']['tags'][0] == 'code':
        
        code_cell = cell['source']
        for code_line in code_cell:
            code_content = code_content + code_line
            if code_line.startswith('import'):
                dependencies.append(code_line.split()[1])

    if cell_type == 'code' and cell['metadata']['tags'][0] == 'plot':
        
        plot_cell = cell['source']
        for plot_line in plot_cell:
            if not (plot_line.startswith('iplot') or plot_line.startswith('from plotly.offline') or plot_line.startswith('init_notebook_mode')):
                plot_content = plot_content + plot_line
                if plot_line.startswith('import'):
                    dependencies.append(plot_line.split()[1])
                        
dep = []
for dependency in dependencies:
    dep.append(dependency.split('.')[0])
    unique_dep = set(dep)

#print(unique_dep)
            
#print('Text is : ', text_content)
#print('Code is : ', code_content)
#print('Plot is : ', plot_content)

markdown_content = "text_content = '''{}'''".format(text_content)

In [35]:
template_content = ''
gitignore_content = ''
procfile_content = ''
template_cells = template_json['cells']
for cell in template_cells:
    cell_type = cell['cell_type']
    if cell_type == 'code':
        template_cell = cell['source']
        for template_line in template_cell:
            template_content = template_content + template_line
    if cell_type == 'markdown' and cell['metadata']['tags'][0] == 'git':
        git_cell = cell['source']
        for git_line in git_cell:
            gitignore_content = gitignore_content + git_line
    if cell_type == 'markdown' and cell['metadata']['tags'][0] == 'proc':
        proc_cell = cell['source']
        for proc_line in proc_cell:
            procfile_content = procfile_content + proc_line

#print(template_content)
#print('  ')
#print(gitignore_content)
#print('   ')
#print(procfile_content)


In [36]:
#creating app folder for organizing application files
!mkdir colaberry-datadoc-app1
%cd colaberry-datadoc-app1

mkdir: cannot create directory ‘colaberry-datadoc-app1’: File exists
/home/mani/workspace/dashframework/colaberry-datadoc-app1


In [37]:
#creating the app.py file, the .gitignore and Procfile for deployment to heroku

fname = 'app.py'
data = code_content + "\n" + markdown_content + "\n" + plot_content  + "\n" + template_content

with open(fname, 'w') as f:
    f.write('{}'.format(data))


with open('.gitignore', 'w') as f:
    f.write('{}'.format(gitignore_content))

with open('Procfile', 'w') as f:
    f.write('{}'.format(procfile_content))

%cd ..

/home/mani/workspace/dashframework


In [6]:
#Actual deployment using deploy.sh
#Requirement - need heroku cli installed and configured locally for this step to work.
args = ''
for each_dep in unique_dep:
    args = args + ' ' + each_dep
!sh deploy.sh {args}


Initialized empty Git repository in /home/mani/workspace/dashframework/colaberry-datadoc-app1/.git/
Running virtualenv with interpreter /home/mani/anaconda3/bin/python3
Using base prefix '/home/mani/anaconda3'
New python executable in /home/mani/workspace/dashframework/colaberry-datadoc-app1/venv/bin/python3
Also creating executable in /home/mani/workspace/dashframework/colaberry-datadoc-app1/venv/bin/python
Installing setuptools, pkg_resources, pip, wheel...done.
  Using cached https://files.pythonhosted.org/packages/25/e6/716d30f51604daedaf12b1064e1d1e48d1c0a224ac41a83496b39862124d/plotly-3.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/bb/a24838832ba35baf52f32ab1a49b906b5f82fb7c76b2f6a7e35e140bac30/decorator-4.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/65/47/7

  Using cached https://files.pythonhosted.org/packages/8c/da/b8dd8deb741bff556db53902d4706774c8e1e67265f69528c14c003644e6/gunicorn-19.9.0-py2.py3-none-any.whl
Uninstalling pkg-resources-0.0.0:
  Successfully uninstalled pkg-resources-0.0.0
 ›   Warning: heroku update available from 7.12.3 to 7.15.0
Creating ⬢ colaberry-datadoc-app1... ⣾ 
⣽⣻⢿⡿⣟⣯⣷⣾⣽⣻⢿⡿⣟⣯⣷Creating ⬢ colaberry-datadoc-app1... done
https://colaberry-datadoc-app1.herokuapp.com/ | https://git.heroku.com/colaberry-datadoc-app1.git
[master (root-commit) bc35f7c] Initial app boilerplate
 4 files changed, 96 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 Procfile
 create mode 100644 app.py
 create mode 100644 requirements.txt
Counting objects: 6, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (6/6), 1.45 KiB | 1.46 MiB/s, done.
Total 6 (delta 0), reused 0 (delta 0)
remote: Compressing source files... done.
remote: Building source:
remote: 
remote: ----

remote:        Collecting dash-core-components==0.29.0 (from -r /tmp/build_571cfb9c09c429e53608272affcb56e6/requirements.txt (line 5))
remote:          Downloading https://files.pythonhosted.org/packages/5c/47/6a1214eee1087cd451a3ad9aa26090845d80bd346953f25663544ff89d8a/dash_core_components-0.29.0.tar.gz (2.1MB)
remote:        Collecting dash-html-components==0.12.0 (from -r /tmp/build_571cfb9c09c429e53608272affcb56e6/requirements.txt (line 6))
remote:          Downloading https://files.pythonhosted.org/packages/94/ec/e7d42a2f48b867a4c8ab448fc794a62b0bcf5ad8bd3b7e7e6c66e54eed61/dash_html_components-0.12.0.tar.gz (42kB)
remote:        Collecting dash-renderer==0.13.2 (from -r /tmp/build_571cfb9c09c429e53608272affcb56e6/requirements.txt (line 7))
remote:          Downloading https://files.pythonhosted.org/packages/bd/14/fa960d38fc68490445a26ad942455ba71ad1140a71c45e7a6b4349bcf922/dash_renderer-0.13.2.tar.gz (160kB)
remote:        Collecting decorator==4.3.0 (from -r /tmp/build_571cfb9c09

remote:            Running setup.py install for retrying: finished with status 'done'
remote:          Running setup.py install for dash-renderer: started
remote:            Running setup.py install for dash-renderer: finished with status 'done'
remote:          Running setup.py install for dash: started
remote:            Running setup.py install for dash: finished with status 'done'
remote:          Running setup.py install for dash-core-components: started
remote:            Running setup.py install for dash-core-components: finished with status 'done'
remote:          Running setup.py install for dash-html-components: started
remote:            Running setup.py install for dash-html-components: finished with status 'done'
remote:        Successfully installed Flask-1.0.2 Flask-Compress-1.4.0 Jinja2-2.10 MarkupSafe-1.0 Werkzeug-0.14.1 certifi-2018.8.24 chardet-3.0.4 click-6.7 dash-0.26.5 dash-core-components-0.29.0 dash-html-components-0.12.0 dash-renderer-0.13.2 decorator-4.3.0 gun

In [27]:
#Redeployment to heroku using redeploy.sh - uncomment the below line for re-deployment
!sh redeploy.sh

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   app.py

no changes added to commit (use "git add" and/or "git commit -a")
[master 940cf19] Updated App
 1 file changed, 6 insertions(+), 6 deletions(-)
Counting objects: 3, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 342 bytes | 342.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Compressing source files... done.
remote: Building source:
remote: 
remote: -----> Python app detected
remote: -----> Installing requirements with pip
remote: 
remote: -----> Discovering process types
remote:        Procfile declares types -> web
remote: 
remote: -----> Compressing...
remote:        Done: 99.5M
remote: -----> Launching...
remote:        Released v6
remote:        https://colaberry-datadoc-app1.herokuapp.com/ deploy

In [7]:
#!python app-gen.py

In [8]:
#from IPython import display

#port = 8050
#url = 'http://127.0.0.1:%d' % port
#iframe = '<iframe src="{url}" width="800" height="800"></iframe>'.format(url=url)
#display.display_html(iframe, raw=True)
